In [1]:
!pip install pillow opencv-python numpy ipywidgets

!pip install ipympl
!pip install opencv-contrib-python


In [2]:
!pip install --user opencv-contrib-python

In [10]:
import cv2
import os
import numpy as np
from PIL import Image, ImageOps
import IPython.display as display

In [2]:
def manual_crop(image_path):
   
    img_cv = cv2.imread(image_path)
    if img_cv is None:
        print("Error: Could not load image from", image_path)
        return None

    while True:
        # Create a resizable window and show the image
        cv2.namedWindow("Select ROI", cv2.WINDOW_NORMAL)
        cv2.imshow("Select ROI", img_cv)
        cv2.waitKey(1)  

        r = cv2.selectROI("Select ROI", img_cv, fromCenter=False, showCrosshair=True)
        cv2.destroyAllWindows()  

      
        if r[2] == 0 or r[3] == 0:
            print("No valid region selected! Try again.")
            retry = input("Do you want to try again? (y/n): ").strip().lower()
            if retry != 'y':
                return None
            continue  

        x, y, w, h = r
        cropped_cv = img_cv[y:y+h, x:x+w]

        cropped_img = Image.fromarray(cv2.cvtColor(cropped_cv, cv2.COLOR_BGR2RGB))
        
        cv2.namedWindow("Cropped Image", cv2.WINDOW_NORMAL)
        cv2.imshow("Cropped Image", cropped_cv)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        output_path = "cropped_image.jpg"
        cropped_img.save(output_path)
        print(f"Cropped image saved as '{output_path}'.")

        return cropped_img  

if __name__ == "__main__":
    image_path = input("Enter the path to your image file: ").strip()
    cropped_img = manual_crop(image_path)
    
    if cropped_img:
        print("Cropping successful!")
    else:
        print("No image was cropped.")

Enter the path to your image file:  C:\\Users\\ektaa\\OneDrive\\Pictures\\Screenshots\\Screenshot (84).png


Cropped image saved as 'cropped_image.jpg'.
Cropping successful!


In [3]:
def auto_crop(image_path, save_path="auto_cropped.jpg"):
    img = cv2.imread(image_path)
    if img is None:
        print("Error: Could not load image from", image_path)
        return None

    saliency = cv2.saliency.StaticSaliencySpectralResidual_create()
    (success, saliencyMap) = saliency.computeSaliency(img)
    if not success:
        print("Error: Saliency detection failed!")
        return None

    saliencyMap = (saliencyMap * 255).astype("uint8")

    _, threshMap = cv2.threshold(saliencyMap, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    contours, _ = cv2.findContours(threshMap.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        print("No contours found; returning original image.")
        return Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)

    cropped = img[y:y+h, x:x+w]

    cv2.imwrite(save_path, cropped)
    print(f"Cropped image saved as '{save_path}'.")

    cropped_pil = Image.fromarray(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))
    return cropped_pil

if __name__ == '__main__':
    image_path = input("Enter the full path to your image file: ").strip()
    cropped_image = auto_crop(image_path)
    if cropped_image:
        cropped_image.show()

Enter the full path to your image file:  C:\\Users\\ektaa\\OneDrive\\Pictures\\Screenshots\\Screenshot (84).png


Cropped image saved as 'auto_cropped.jpg'.


In [4]:
def add_margin(image, margin_size=None, margin_color=(255, 255, 255)):
    width, height = image.size

    if margin_size is None:
        margin_size = int(0.05 * min(width, height))
        print("Using automatic margin size:", margin_size)

    new_size = (width + 2 * margin_size, height + 2 * margin_size)

    new_img = Image.new("RGB", new_size, margin_color)

    new_img.paste(image, (margin_size, margin_size))
    
    return new_img

if __name__ == '__main__':
    image_path = input("Enter the path to your image file: ").strip()
    image = Image.open(image_path)

    choice = input("Do you want to specify a margin size? (y/n): ").strip().lower()
    if choice == "y":
        margin = int(input("Enter margin size in pixels: "))
    else:
        margin = None  
    
    result_img = add_margin(image, margin)
    result_img.show()

    result_img.save("image_with_margin.jpg")
    print("Image with margin saved as 'image_with_margin.jpg'.")

Enter the path to your image file:  C:\\Users\\ektaa\\OneDrive\\Pictures\\Screenshots\\Screenshot (84).png
Do you want to specify a margin size? (y/n):  n


Using automatic margin size: 38
Image with margin saved as 'image_with_margin.jpg'.


In [12]:
def resize_image(image, target_width, aspect_ratio_choice="original", custom_ratio=None):
    if aspect_ratio_choice == "original":
        width, height = image.size
        new_height = int(target_width * height / width)
    elif aspect_ratio_choice == "16:9":
        new_height = int(target_width * 9 / 16)
    elif aspect_ratio_choice == "4:3":
        new_height = int(target_width * 3 / 4)
    elif aspect_ratio_choice == "1:1":
        new_height = target_width 
    elif aspect_ratio_choice == "custom" and custom_ratio is not None:
        w_ratio, h_ratio = custom_ratio
        new_height = int(target_width * h_ratio / w_ratio)
    else:
        width, height = image.size
        new_height = int(target_width * height / width)
    
    resized_img = image.resize((target_width, new_height), Image.LANCZOS)
    return resized_img

if __name__ == '__main__':
    image_path = input("Enter the path to your image file: ").strip()
    target_width = int(input("Enter target width (in pixels): "))
    
    print("Choose the desired aspect ratio:")
    print("1. Original")
    print("2. 16:9")
    print("3. 4:3")
    print("4. 1:1 (Square)")
    print("5. Custom")
    choice = input("Enter your choice (1-5): ").strip()

    aspect_ratio = "original"
    custom_ratio = None
    if choice == "1":
        aspect_ratio = "original"
    elif choice == "2":
        aspect_ratio = "16:9"
    elif choice == "3":
        aspect_ratio = "4:3"
    elif choice == "4":
        aspect_ratio = "1:1"
    elif choice == "5":
        aspect_ratio = "custom"
        custom_input = input("Enter custom ratio as width:height (e.g., 3:2): ")
        try:
            w_str, h_str = custom_input.split(":")
            custom_ratio = (float(w_str), float(h_str))
        except Exception as e:
            print("Invalid custom ratio. Defaulting to original aspect ratio.")
            aspect_ratio = "original"
    else:
        print("Invalid choice. Defaulting to original aspect ratio.")
    
    image = Image.open(image_path)

    resized_image = resize_image(image, target_width, aspect_ratio, custom_ratio)
 
    resized_image.show()

    resized_image = resized_image.convert("RGB")


    
    output_path = "resized_image.jpg"
    resized_image.save(output_path)
    print(f"Resized image saved as '{output_path}'.")


Enter the path to your image file:  C:\\Users\\ektaa\\OneDrive\\Pictures\\Screenshots\\Screenshot (84).png
Enter target width (in pixels):  10000


Choose the desired aspect ratio:
1. Original
2. 16:9
3. 4:3
4. 1:1 (Square)
5. Custom


Enter your choice (1-5):  3


Resized image saved as 'resized_image.jpg'.
